# Coding with Jupyter AI 

This notebook provides step-by-step instructions for replicating what Andrew did in the first demo. Please follow along, and feel free to play with different variations too! 


---
<a name='Notes'></a>

<h4 style="color:green; font-weight:bold;">Notes:</h4>

* All the required Python packages have been installed in this environment.
* Your generated code might look different from the code shown in the video due to LLM stochasticity.
* You can collapse the course menu on the left by clicking this icon: 
  
  <img src="./images/file_collapse_bordered.png" width="200" style="vertical-align: middle;">
* To open Jupyter chat, click on the chat bubble icon on the left sidebar of Jupyter Lab:
  
  <img src="images/jupyter_chat_bordered.png" width="150" style="vertical-align: middle;">

  
* The exercises are not graded (you won't see a green tick next to each exercise item). You will receive 100% completion by watching the videos and marking the reading item (located before the conclusion) as complete.  

* If there's no activity for 25 minutes in the notebook, the environment will automatically reset. You might notice the chat window stops responding. If this happens, refresh the webpage to start again.

---

## Step 1: Print "Hello World"

- Open Jupyter chat by clicking on the chat bubble icon on the left sidebar of Jupyter Lab
- In the Jupyter chat interface, create a new chat by clicking `+Chat`
  - Name it anything you'd like (e.g., `Chat 1`)
- Use a prompt like this:
   > Write a code cell to print Hello World
- Transfer the generated code to the cell below, and run it

In [1]:
print("Hello World")

Hello World


## Step 2: Generate a Fun Cookie Fortune Message 

- Create a new chat by clicking `+Chat` (`Chat 2`)
- Use a prompt like this:
  > Create a code cell that uses OpenAI’s gpt-4.1-mini model to write a fun cookie fortune message for the user. The program should first ask the user to input their name, then generate the message and print it out.
  > 
  > To call gpt-4.1-mini, use python-dotenv to load the secret OPENAI_API_KEY from .env. Then, use this prompt to call the model:
  > 
  > """Write a brief and humorous cookie fortune message for {name} (1 sentence)."""
- Transfer the generated code to the cell below, and run it

<span style="color:green; font-weight:bold;">Note:</span> The `OPENAI_API_KEY` variable is already defined in this environment, you do not need to create  an `.env` file. 

In [2]:
# Import required packages
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Ask the user for their name
name = input("Enter your name: ")

# Define the prompt
prompt = f"Write a brief and humorous cookie fortune message for {name} (1 sentence)."

# Generate the cookie fortune message
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

# Extract and print the message
fortune = response.choices[0].message.content.strip()
print("\n🍪 Your Cookie Fortune 🍪")
print(fortune)

Enter your name:  hello



🍪 Your Cookie Fortune 🍪
Hello! Your future is as bright as your browser history—mostly interesting and a little mysterious.


## Step 3: Understand Code for Data Analysis

- Run the cell below to load the customer reviews into a Pandas DataFrame

In [4]:
import pandas as pd

df = pd.read_csv("data/customer_reviews.csv")
df.head(5)

,product_id,customer_id,rating,review_text,review_date,product_category,verified_purchase
0,P001,C1547,5,Absolutely love this product! Exceeded all my ...,2024-01-15,Electronics,True
1,P001,C2891,4,Great product overall. Works as advertised. On...,2024-01-18,Electronics,True
2,P002,C3456,1,Very disappointed. Stopped working after just ...,2024-01-20,Electronics,True
3,P002,C4123,2,Not worth the price. Quality feels cheap and i...,2024-01-22,Electronics,False
4,P003,C5678,5,Perfect! Exactly what I needed. The design is ...,2024-01-25,Home & Kitchen,True


- Run the cell below to display the analysis
- Create a new chat by clicking `+Chat` (`Chat 3`)
- Drag and drop the code cell below to the chat interface, and use a prompt like this:
  > What does this code cell do? (starting with creating sentiment_category column)

In [5]:
# Create sentiment categories based on numeric rating
df['sentiment_category'] = pd.cut(
    df['rating'],
    bins=[0, 2, 3, 5],
    labels=['Negative', 'Neutral', 'Positive']
)

# Group by category and sentiment, aggregating key metrics
analysis = (
    df.groupby(['product_category', 'sentiment_category'])
      .agg(
          rating_count=('rating', 'count'),
          product_id_nunique=('product_id', 'nunique')
      )
      .reset_index()
)

# Fill NaNs resulting from empty groups with zeros
analysis = analysis.fillna({
    'rating_count': 0,
    'product_id_nunique': 0
})

# Calculate percentage of negative reviews safely
category_counts = df.groupby('product_category').size()
negative_counts = (
    df[df['sentiment_category'] == 'Negative']
    .groupby('product_category')
    .size()
)
negative_pct = (negative_counts / category_counts * 100).fillna(0)

# Display results
print("Review analysis by category and sentiment:")
print(analysis)
print("\nPercentage of negative reviews by category:")
print(negative_pct.round(1))

Review analysis by category and sentiment:
   product_category sentiment_category  rating_count  product_id_nunique
0             Books           Negative             2                   1
1             Books            Neutral             0                   0
2             Books           Positive             2                   1
3          Clothing           Negative             1                   1
4          Clothing            Neutral             1                   1
5          Clothing           Positive             5                   3
6       Electronics           Negative             5                   3
7       Electronics            Neutral             0                   0
8       Electronics           Positive             6                   4
9   Health & Beauty           Negative             0                   0
10  Health & Beauty            Neutral             1                   1
11  Health & Beauty           Positive             5                   4
12   Hom

/tmp/ipykernel_24/3719147239.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['product_category', 'sentiment_category'])


## Step 4: Summarize Customer Reviews

- In the same chat you created in the previous step, drag and drop the Markdown cell below and use a prompt like this:
  > Follow the instructions in the Markdown cell to generate code (3 notebook cells) to analyze the data.
___

Create code that extracts reviews from a Pandas DataFrame and summarizes the issues reported by customers. Create a code cell for each step:

**Step 1: Extract the reviews in a list**

The reviews are saved in the column `review_text` of the DataFrame `df`. Extract all reviews in a list `combined_reviews`.

**Step 2: Examine the reviews** 

Use OpenAI’s gpt-4.1-mini model to summarize the content in the reviews. The goal is to output a summary of product and service issues. To call gpt, assume the OPENAI_API_KEY is defined in .env, and use python-dotenv to load it. Use this prompt:

""" 
You are analyzing customer reviews to identify product and service issues.

Below are customer reviews for a set of products. Please analyze them and provide:

1. **Common Product Issues**: List the main problems customers reported about the product itself (quality, functionality, features, etc.)
2. **Common Service Issues**: List the main problems customers reported about the service (shipping, customer support, packaging, etc.)
3. **Frequency**: For each issue, estimate how often it appears (e.g., "mentioned frequently", "occasional complaint", "rare issue")

Format your response clearly with headers and bullet points.

CUSTOMER REVIEWS:
{combined_reviews} 

"""

**Step 3: Print & Save the summary** 

Print the summary and save it as a markdown file `customer_reviews.md`


___

In [6]:
# Step 1: Extract reviews from the DataFrame

# Assuming your DataFrame is named df and has a column 'review_text'
combined_reviews = df['review_text'].dropna().tolist()

# Preview the first few reviews
combined_reviews[:5]

['Absolutely love this product! Exceeded all my expectations. The quality is outstanding and it arrived faster than promised. Will definitely buy again!',
 'Great product overall. Works as advertised. Only minor issue is the instructions could be clearer, but figured it out eventually.',
 'Very disappointed. Stopped working after just 2 weeks. Customer service was unhelpful. Would not recommend.',
 "Not worth the price. Quality feels cheap and it doesn't perform as well as described. Returning it.",
 "Perfect! Exactly what I needed. The design is beautiful and it's very user-friendly. Highly recommend!"]

In [7]:
# Step 2: Use OpenAI API to summarize reviews

import os
from dotenv import load_dotenv
from openai import OpenAI

# Load the API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Define the analysis prompt
prompt = f"""
You are analyzing customer reviews to identify product and service issues.

Below are customer reviews for a set of products. Please analyze them and provide:

Common Product Issues: List the main problems customers reported about the product itself (quality, functionality, features, etc.)
Common Service Issues: List the main problems customers reported about the service (shipping, customer support, packaging, etc.)
Frequency: For each issue, estimate how often it appears (e.g., "mentioned frequently", "occasional complaint", "rare issue")
Format your response clearly with headers and bullet points.

CUSTOMER REVIEWS: {combined_reviews}
"""

# Call OpenAI’s gpt-4.1-mini model
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a helpful data analyst."},
        {"role": "user", "content": prompt}
    ]
)

# Extract the summary text
summary = response.choices[0].message.content

In [8]:
# Step 3: Print and save the summary

# Print the model-generated summary
print(summary)

# Save the summary as a Markdown file
with open("customer_reviews.md", "w", encoding="utf-8") as f:
    f.write(summary)

**Common Product Issues**

- Poor quality and durability  
  - Mentioned frequently  
  - Complaints about products breaking on first use or within a short time (e.g., "Stopped working after just 2 weeks," "Broke on first use," "Feels like it will break any moment")  
  - Described as cheaply made or using poor materials  

- Misleading product descriptions and photos  
  - Mentioned occasionally to frequently  
  - Customers report product not matching photos or description (e.g., "Not as described," "Very misleading," "Product description was very misleading")  

- Limited features or underwhelming performance  
  - Occasional complaint  
  - Some users felt product was average or lacked expected features ("Does the job but nothing special," "Expected more features")  

- Instruction or design flaws  
  - Occasional complaint  
  - Minor issues with unclear instructions or small design flaws impacting usability ("Instructions could be clearer," "Minor design flaw")  

- Mixed/conflic